<style>
    pre {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
</style>

<div style="display:flex; justify-content:space-around; align-items:center; background-color:#cccccc; padding:5px; border:2px solid #333333;">
    <a href="https://www.um.es/web/estudios/grados/ciencia-ingenieria-datos/" target="_blank">
    <img src="https://www.um.es/documents/1073494/42130150/LogosimboloUMU-positivo.png" alt="UMU" style="height:200px; width:auto;">
    <a href="https://estudios.upct.es/grado/5251/inicio" target="_blank">
    <img src="https://www.upct.es/contenido/universidad/galeria/identidad-2021/logos/logos-upct/marca-upct/marca-principal/horizontal/azul.png" alt="UPCT" style="height:145px; width:auto;">
</div>

# Asignatura: **Deep Learning**

## Titulación: **Grado en Ciencia e Ingeniería de Datos**

## Práctica 3: Transformers
### **Sesión 2/3: Redes recurrentes y embedding**

**Autores**: Antonio Martínez Sánchez, Juan Morales Sánchez, José Luís Sancho Gómez y Juan Antonio Botía Blaya

<div style="page-break-before: always;"></div>

### Contenidos
- [Requisitos](#requisitos)
- [Procesamiento de secuencias](#secuencias)
- [Redes recurrentes](#rnn)
- [Embedding](#embedding)
- [Ejercicios](#ejercicios)

### Requisitos
<a class='anchor' id='requisitos'></a>

Se trabajará con notebooks de [Jupyter](https://jupyter.org/install) con código Python empleando como intérprete la última versión de [Miniconda](https://docs.anaconda.com/miniconda/). Se requiere la preinstalación (se recomienda utilizar [pip](https://pypi.org/project/pip/)) de los siguientes paquetes de Python:

- [Numpy](https://pypi.org/project/numpy/) (computación numérica)
- [Tensorflow](https://www.tensorflow.org/install/pip?hl=es-419#linux) que incluye a Keras (deep learning)
- [Scikit-learn](https://pypi.org/project/scikit-learn/) (machine learning)
- [Matplotlib](https://pypi.org/project/matplotlib/) (visualización de datos)
- [Pandas](https://pypi.org/project/seaborn/) (manipulación de datos tabulados)

### Procesamiento de secuencias
<a class='anchor' id='secuencias'></a>

Una de las limitaciones de las aproximaciones utilizadas en la Sesión 1 de esta práctica es que tratan los fragmentos de texto simplemente como un conjunto de tokens (*bag-of-words*) y no como secuencias, siendo por tanto insensibles al orden los tokens en un texto. En la mayoría de los casos la relación de un token con sus vecinos condiciona su semántica. La información adicional proporcionada por la relación de una palabra (token) con sus vecinas permite a estos modelos obtener mejores resultados cuando las condiciones son las adecuadas.

### Redes recurrentes
<a class='anchor' id='rnn'></a>

Las redes neuronales recurrentes (RNNs) se desarrollaron para procesar series temporales en la que una magnitud varía en función del tiempo. El texto se puede transformar en una secuencia temporal codificando cada token y asignándole a su posición en la secuencia un valor temporal. El problema de las primeras RNNs era el desvanecimiento de gradiente que aparecía cuando se querían procesar secuencias largas. Para solventar este problema surgió la arquitectura *long short-term memory* (LSTM) que se basa en células de memoria que pueden retener información durante periodos arbitrarios. Además, en nuestro problema en concreto necesitamos procesar las secuencias en las dos posibles direcciones, ya que no podemos asumir que un AA en un páptido solo dependa de los AAs anteriores, también podría venir influenciado por los AAs posteriores. El esquema de la red LSTM bidireccional que vamos a construir es el siguiente:

<img src="./figs/bi_lstm.png" width="400">

Keras dispone de dos classes que permitirán construir fácilmente esta red RNN, la clase [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/) que construye una capa de celular LSTM y la clase [Bidirectional](https://keras.io/api/layers/recurrent_layers/bidirectional/) que pasa las secuencias en los dos direcciones para finalmente fusionar la salida. 

### Embedding
<a class='anchor' id='embedding'></a>

Cuando se realiza una codificación directa sin analizar los datos como one-hot, que a cada token le asigna un vector con tantas dimensiones como palabras tiene el lenguaje con todos los valores a 0 excepto un 1 en el índice de la palabra correspondiente, en realidad se está asumiendo que todos los tokens son independientes los unos de los otros. En one-hot los vectores que se generan son ortonormales entre sí. En general las palabras (tokens) de un lenguaje forman un espacio estructurado: comparten información entre ellas. Las palabras "palabra" y "token" son intercambiables en la mayoría de los casos de este documento, luego un vector que represente a "palabra" no debería ser ortonormal del que representa a "token", estos vectores deberían ser muy similares. Con *embedding* nos referimos a obtener una representación que haga precisamente esto: mapear las palabras de un lenguaje en un espacio geométrico estructurado. La clase [Embedding](https://keras.io/api/layers/core_layers/embedding/) del paquete Keras permite crear una capa para hacer un embedding de los datos de entrada a una red de forma sencilla.

Mientras que los vectores obtenidos por una codificación one-hot son dispersos (muchos bits para representar poca información), binarios y tienen una dimensión que depende del tamaño del vocabulario de un lenguaje. El embedding permite generar vectores densos, con una dimensionalidad arbitraria y que se representan una estructura aprendida de los datos. Palabras similares aparecen próximas en el espacio embebido, es más, aparecerán direcciones con un significado semántico específico. Por ejemplo, en la gráfica de abajo se muestra se muestran cuatro palabras embebidas en un plano 2D: "gato", "perro", "lobo" y "tigre". En este espacio relaciones semánticas se han codificado en transformaciones geométricas. El mismo vector que nos permite movernos de "gato" a "tigre", también nos llevaría de "perro" a "lobo", luego este vector se podría interpretar como la transformación: de animal doméstico a salvaje. De manera similar, el vector que nos lleva de "perro" a "gato" o de "lobo" a "tigre" se podría interpretar como la transformación: de canino a felino. 

<img src="./figs/embedding.png" width="400">

En la práctica los lenguajes suelen ser tan complejos que los embeddings que los representar suelen tener muchas dimensiones. Para analizar visualmente estos espacios se requiere de técnicas de reducción de la dimensionalidad, dónde PCA (Principal Components Analysis) sea quizás la más conocida. Estas técnicas permiten proyectar un espacio de muchas dimensiones en un plano 2D minimizando la información pérdida y facilianado su análisis mediante inspección visual. El paquete *scikit-learn* contiene la clase [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) con una implementación del algoritmo PCA. 

### Ejercicios
<a class='anchor' id='ejercicios'></a>

**E1:** Codifica el dataset de la Sesión 1 utilizando esta vez una codificación de tipo *integer*.

**E2:** Construye un modelo LSTM bidireccional con 50 células LSTM, entrénalo empleando el resultado de E1 y evalúa la precisión del modelo entrenado. Para el entrenamiento utiliza la aproximación empleada para los ejercicios de la Sesión 1.

**E3:** Repite el ejercicio anterior, pero utilizando una codificación *one-hot*. ¿Qué ventaja ofrece la codificación *one-hot* sobre *integer*? ¿Se podría utilizar la codificación *one-hot* para procesar lenguaje natural? ¿por qué?

**E4:** Repite el ejercicio anterior, pero utilizando *embedding* de 16 dimensiones sobre la codificación *integer*. ¿Mejoran los resultados? ¿por qué? ¿Ayuda activar la opción *mask_zero* de la capa de embedding? ¿por qué? 

**E5:** Realiza varias repeticiones del entrenamiento de los modelos en los E2-3. ¿Obtienes siempre los mismos resultados? ¿por qué? ¿qué modelo presenta un comportamiento más estable?

**E6:** Representa en un plano 2D los diferentes AAs sobre espacio generados por el embedding. ¿Pon un ejemplo de dos AAs con una funcionalidad muy similar? ¿qué AAs parecen tener un comportamiento diferente al resto?